In [7]:
import pandas as pd
import requests
from tqdm import tqdm
from pathlib import Path
import os

In [8]:
os.getcwd()

'/Users/colleenking/Documents/college/programming_for_data_analytics/project'

In [10]:
data_dir = Path("./data")

if not data_dir.exists():
    os.mkdir(data_dir)
    os.mkdir(data_dir / "weather_data")

Read in the details of all weather stations - some lines aren't parsed correctly so these are skipped

We get the county, station name, the station id (station name), the height above sea level, location both in easting/northing and latitude/longitude, the open and close years

In [24]:
station_details_url = "http://cli.fusio.net/cli/climate_data/webdata/StationDetails.csv"

station_df = pd.read_csv(station_details_url, on_bad_lines='skip')

station_df.head()

,county,station name,name,height(m),easting,northing,latitude,longitude,open year,close year
0,Antrim,5880,LH_RATHLIN_WEST,10,309200,451800,55.30083,-6.28028,2000,(null)
1,Carlow,4415,TULLOW (Waterworks),76,284700,173400,52.80528,-6.74306,1985,(null)
2,Carlow,2414,BORRIS G.S.,85,272400,150700,52.60278,-6.93056,1944,1991
3,Carlow,1214,CARLOW (SUGAR FACTORY),58,272200,178400,52.85139,-6.92778,1953,1960
4,Carlow,115,HACKETSTOWN RECTORY,182,297600,180500,52.86667,-6.55000,1910,1944


In [14]:
station_df.dtypes

county           object
station name      int64
name             object
height(m)         int64
easting           int64
northing          int64
latitude        float64
longitude       float64
open year        object
close year       object
dtype: object

The open and close years are stored as stings, but we want them as ints

In [26]:
station_df["close year"] = station_df["close year"].replace('(null)', None).astype('Int64')
station_df["open year"] = station_df["open year"].replace('(null)', None).astype('Int64')
station_df.head()

,county,station name,name,height(m),easting,northing,latitude,longitude,open year,close year
0,Antrim,5880,LH_RATHLIN_WEST,10,309200,451800,55.30083,-6.28028,2000,<NA>
1,Carlow,4415,TULLOW (Waterworks),76,284700,173400,52.80528,-6.74306,1985,<NA>
2,Carlow,2414,BORRIS G.S.,85,272400,150700,52.60278,-6.93056,1944,1991
3,Carlow,1214,CARLOW (SUGAR FACTORY),58,272200,178400,52.85139,-6.92778,1953,1960
4,Carlow,115,HACKETSTOWN RECTORY,182,297600,180500,52.86667,-6.55000,1910,1944


In [27]:
station_df.dtypes

county           object
station name      int64
name             object
height(m)         int64
easting           int64
northing          int64
latitude        float64
longitude       float64
open year         Int64
close year        Int64
dtype: object

In [28]:
station_df.to_csv(data_dir / "stations.csv", index=False)

We can use the station id to download daily data automatically because the download url is predictable - replacing the station id gives the filename we're looking for (https://cli.fusio.net/cli/climate_data/webdata/dly[station_id].csv)

We also don't care about closed stations, so we only need the rows where close year is null

In [64]:
open_stations = list(station_df[~pd.isna(station_df["close year"])]["station name"])
len(open_stations)

1550

1550 is a lot of stations - each file is about 2mb, so in total this would be around 3Gb if we naively downloaded it all. We should just get a sample a few from each county instead.

5 stations * 26 counties * 2mb gives around 260mb, which is much more reasonable

In [39]:
# https://stackoverflow.com/questions/22472213/python-random-selection-per-group

sample_stations = station_df[~pd.isna(station_df["close year"])].groupby('county').apply(lambda x: x.sample(5)).reset_index(drop=True)

print(len(sample_stations))
sample_stations.head()


130


/var/folders/s7/bsy4w2n54pvcvlbx9jg9h2040000gn/T/ipykernel_13248/2311088952.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_stations = station_df[~pd.isna(station_df["close year"])].groupby('county').apply(lambda x: x.sample(5)).reset_index(drop=True)


,county,station name,name,height(m),easting,northing,latitude,longitude,open year,close year
0,Carlow,6014,CLASHGANNA MILLS,27,273800,146200,52.56250,-6.91111,1987,1989
1,Carlow,4814,CARLOW (Oak Park),61,273000,179500,52.86111,-6.91528,1967,1996
2,Carlow,515,TULLOW (MT.ST.JOSEPH'S),79,285800,171800,52.79028,-6.72778,1949,1960
3,Carlow,5314,BAGENALSTOWN (FENAGH II),104,277700,161500,52.69917,-6.85111,1982,1989
4,Carlow,6914,GARRYHILL (MILLTOWN),107,278600,158700,52.67389,-6.83750,2003,2005


In [41]:
# url format: https://cli.fusio.net/cli/climate_data/webdata/dly875.csv
for station in tqdm(list(sample_stations['station name'])):
    try:
        url = f"https://cli.fusio.net/cli/climate_data/webdata/dly{station}.csv"
        response = requests.get(url)
        assert response.status_code == 200
        with open(data_dir / f"weather_data/{station}.csv", "wb") as f:
            f.write(response.content)
    except:
        print(f"Couldn't retrieve data for station {station}")

  2%|▋                                          | 2/130 [00:00<00:23,  5.44it/s]

Couldn't retrieve data for station 6014
Couldn't retrieve data for station 4814


  3%|█▎                                         | 4/130 [00:00<00:21,  5.92it/s]

Couldn't retrieve data for station 515
Couldn't retrieve data for station 5314


  5%|█▉                                         | 6/130 [00:01<00:21,  5.80it/s]

...

Couldn't retrieve data for station 508
Couldn't retrieve data for station 1520


 98%|████████████████████████████████████████▎| 128/130 [00:24<00:00,  6.19it/s]

Couldn't retrieve data for station 2515
Couldn't retrieve data for station 1824


100%|█████████████████████████████████████████| 130/130 [00:24<00:00,  5.32it/s]

Couldn't retrieve data for station 3424
Couldn't retrieve data for station 815


It looks like a lot of the stations' data isn't available via these urls - of the 130 expected, only 7 were actually downloaded, or a little over 5%. If that holds true across the whole dataset, 5% of the 1550 stations should give us an expected 83 files, totalling ~160mb, which would be manageable.

In [44]:
1550*(7/130)

83.46153846153847

In [42]:
# this station gives a 404 Not Found status code - the file doesn't exist at this url
pd.read_csv(f"https://cli.fusio.net/cli/climate_data/webdata/dly6014.csv")

HTTPError: HTTP Error 404: Not Found

In [45]:
files_downloaded = 0
for station in tqdm(open_stations):
    try:
        url = f"https://cli.fusio.net/cli/climate_data/webdata/dly{station}.csv"
        response = requests.get(url)
        assert response.status_code == 200
        with open(data_dir / f"weather_data/{station}.csv", "wb") as f:
            f.write(response.content)
        files_downloaded += 1
    except:
        pass
print(f'Downloaded {files_downloaded} files')

100%|███████████████████████████████████████| 1550/1550 [03:15<00:00,  7.94it/s]

Downloaded 38 files


We only got 38 files in the end, we may have gotten lucky with the sample that had a hit rate of 5%.

Now that we have all our files downloaded, we need to read them into pandas so we can analyse them.
Each file has a header that we need to skip, however the number of rows changes from file to file. The row we want always starts with "date", so we can use this.

We also need to keep track of the station id so we can match it with the station information we have above.

In [49]:
# https://stackoverflow.com/questions/68369338/how-to-read-a-csv-in-python-using-pandas-read-csv-after-a-certain-line-having-s

from io import StringIO
import copy

# need to skip 8
with open('data/weather_data/405.csv', 'r') as f:
    file1 = f.read()

# need to skip 12
with open('data/weather_data/5911.csv', 'r') as f:
    file2 = f.read()

def read_csv_after_line(file, str_to_find):
    iter_file = copy.copy(file)
    for i,line in enumerate(iter_file):
        if str_to_find.lower() in line.lower():
            print(i,line)
            return pd.read_csv(file, skiprows=i)

df_1 = read_csv_after_line(StringIO(file1),'date,ind')

df_2 = read_csv_after_line(StringIO(file2),'date,ind')


9 date,ind,rain

13 date,ind,rain,ind,maxt,ind,mint,gmin,soil



In [53]:
df_1.tail()

,date,ind,rain
24125,26-jun-2016,0,0.0
24126,27-jun-2016,0,1.5
24127,28-jun-2016,0,3.8
24128,29-jun-2016,0,2.9
24129,30-jun-2016,0,3.0


In [52]:
df_2.tail()

,date,ind,rain,ind.1,maxt,ind.2,mint,gmin,soil
9001,24-feb-2019,0,0.0,0,11.0,0,8.0,,
9002,25-feb-2019,0,0.0,0,14.6,0,8.4,,
9003,26-feb-2019,0,0.0,0,14.4,0,4.8,,
9004,27-feb-2019,0,0.0,0,14.4,0,4.0,,
9005,28-feb-2019,0,0.0,0,13.5,0,6.0,,


In [57]:
data = {}

for file in os.listdir('data/weather_data/'):
    if '.csv' in file:
        with open(f'data/weather_data/{file}', 'r') as f:
            raw_file = f.read()
            
        data[int(file.replace('.csv', ''))] = read_csv_after_line(StringIO(raw_file),'date,ind')

9 date,ind,rain

13 date,ind,rain,ind,maxt,ind,mint,gmin,soil

9 date,ind,rain

9 date,ind,rain

9 date,ind,rain

13 date,ind,rain,ind,maxt,ind,mint,gmin,soil

9 date,ind,rain

13 date,ind,rain,ind,maxt,ind,mint,gmin,soil

9 date,ind,rain

13 date,ind,rain,ind,maxt,ind,mint,gmin,soil

9 date,ind,rain

13 date,ind,rain,ind,maxt,ind,mint,gmin,soil

9 date,ind,rain

9 date,ind,rain

9 date,ind,rain

13 date,ind,rain,ind,maxt,ind,mint,gmin,soil

9 date,ind,rain

9 date,ind,rain

13 date,ind,rain,ind,maxt,ind,mint,gmin,soil

9 date,ind,rain

9 date,ind,rain

9 date,ind,rain

9 date,ind,rain

9 date,ind,rain

9 date,ind,rain

9 date,ind,rain

13 date,ind,rain,ind,maxt,ind,mint,gmin,soil

13 date,ind,rain,ind,maxt,ind,mint,gmin,soil

9 date,ind,rain

13 date,ind,rain,ind,maxt,ind,mint,gmin,soil

9 date,ind,rain

9 date,ind,rain

9 date,ind,rain

9 date,ind,rain

9 date,ind,rain

9 date,ind,rain

13 date,ind,rain,ind,maxt,ind,mint,gmin,soil

13 date,ind,rain,ind,maxt,ind,mint,gmin,soil

9 date

None of these stations have wind speed as a metric - but this is because we've mistakenly downloaded the closed station data instead of the open stations - which also explains why so few of them have publically available data. We know that Mullingar has data and the metric we're looking for, but it's not in the list of open stations.

In [58]:
station_df[station_df['station name'] == 875]

,county,station name,name,height(m),easting,northing,latitude,longitude,open year,close year
1851,Westmeath,875,MULLINGAR,101,243000,254300,53.53722,-7.36222,2002,<NA>


In [59]:
875 in open_stations

False

There are actually 530 stations rather than 1550. So we can go back to our original strategy of sampling by county.

In [65]:
open_stations = list(station_df[pd.isna(station_df["close year"])]["station name"])
len(open_stations)

530

In [71]:
sample_stations = station_df[pd.isna(station_df["close year"])].groupby('county').apply(lambda x: x.sample(min(len(x), 5))).reset_index(drop=True)

print(len(sample_stations))
sample_stations.head()


129


/var/folders/s7/bsy4w2n54pvcvlbx9jg9h2040000gn/T/ipykernel_13248/3599114514.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_stations = station_df[pd.isna(station_df["close year"])].groupby('county').apply(lambda x: x.sample(min(len(x), 5))).reset_index(drop=True)


,county,station name,name,height(m),easting,northing,latitude,longitude,open year,close year
0,Antrim,5880,LH_RATHLIN_WEST,10,309200,451800,55.30083,-6.28028,2000,<NA>
1,Carlow,375,OAK PARK,62,273000,179500,52.86111,-6.91528,2003,<NA>
2,Carlow,6114,POLLMOUNTY FISH FARM,24,274500,135545,52.46667,-6.90278,1987,<NA>
3,Carlow,4415,TULLOW (Waterworks),76,284700,173400,52.80528,-6.74306,1985,<NA>
4,Carlow,4515,TULLOW (Ardoyne Glebe),79,288200,169800,52.77194,-6.69333,1985,<NA>


We were able to download 128 of the 129 target files this time. However many of them still do not have wind speed as a metric, so we'll have to ignore these stations.

In [73]:
files_downloaded = 0
for station in tqdm(list(sample_stations['station name'])):
    try:
        url = f"https://cli.fusio.net/cli/climate_data/webdata/dly{station}.csv"
        response = requests.get(url)
        assert response.status_code == 200
        with open(data_dir / f"weather_data/{station}.csv", "wb") as f:
            f.write(response.content)
        files_downloaded += 1
    except:
        pass
print(f'Downloaded {files_downloaded} files')

100%|█████████████████████████████████████████| 129/129 [00:44<00:00,  2.90it/s]

Downloaded 128 files


In [74]:
data = {}

for file in os.listdir('data/weather_data/'):
    if '.csv' in file:
        with open(f'data/weather_data/{file}', 'r') as f:
            raw_file = f.read()
            
        data[int(file.replace('.csv', ''))] = read_csv_after_line(StringIO(raw_file),'date,ind')

9 date,ind,rain

9 date,ind,rain

9 date,ind,rain

9 date,ind,rain

9 date,ind,rain

9 date,ind,rain

13 date,ind,rain,ind,maxt,ind,mint,gmin,soil

24 date,ind,maxtp,ind,mintp,igmin,gmin,ind,rain,cbl,wdsp,ind,hm,ind,ddhm,ind,hg,soil,pe,evap,smd_wd,smd_md,smd_pd,glorad

13 date,ind,rain,ind,maxt,ind,mint,gmin,soil

9 date,ind,rain

9 date,ind,rain

9 date,ind,rain

13 date,ind,rain,ind,maxt,ind,mint,gmin,soil

13 date,ind,rain,ind,maxt,ind,mint,gmin,soil

9 date,ind,rain

9 date,ind,rain

9 date,ind,rain

13 date,ind,rain,ind,maxt,ind,mint,gmin,soil

9 date,ind,rain

9 date,ind,rain

9 date,ind,rain

9 date,ind,rain

9 date,ind,rain

9 date,ind,rain

9 date,ind,rain

9 date,ind,rain

9 date,ind,rain

9 date,ind,rain

9 date,ind,rain

13 date,ind,rain,ind,maxt,ind,mint,gmin,soil

24 date,ind,maxtp,ind,mintp,igmin,gmin,ind,rain,cbl,wdsp,ind,hm,ind,ddhm,ind,hg,soil,pe,evap,smd_wd,smd_md,smd_pd,glorad

9 date,ind,rain

9 date,ind,rain

13 date,ind,rain,ind,maxt,ind,mint,gmin,soil

9 date,in

In [76]:
wind_data = {station: df for station, df in data.items() if 'wdsp' in df.columns}
len(wind_data.items())

7

In [78]:
for x, y in wind_data.items():
    print(len(y.columns))

24
24
26
24
24
24
24


In the end, only 7 of the downloaded files have the metric we're looking for.

In [85]:
dfs = []
for station, df in wind_data.items():
    df['station name'] = station
    dfs.append(df)

combined_df = pd.concat(dfs)
combined_df.head()


,date,ind,maxtp,ind.1,mintp,igmin,gmin,ind.2,rain,cbl,...,pe,evap,smd_wd,smd_md,smd_pd,glorad,station name,sun,dos,g_rad
0,19-sep-2008,0,16.8,0,11.7,,,0,0.0,1019.5,...,,,,,,1063,1775,NaN,NaN,NaN
1,20-sep-2008,0,17.0,0,10.4,,,0,0.0,1021.1,...,,,,,,1525,1775,NaN,NaN,NaN
2,21-sep-2008,0,17.0,0,9.6,,,0,0.2,1019.4,...,,,,,,1340,1775,NaN,NaN,NaN
3,22-sep-2008,0,16.0,0,10.6,,,0,0.0,1021.4,...,,,,,,1572,1775,NaN,NaN,NaN
4,23-sep-2008,0,14.5,0,8.1,,,0,0.0,1021.2,...,,,,,,1468,1775,NaN,NaN,NaN


Using a similar process to assignment 6, we can clean up this combined dataset. We can also trim out some of the columns we don't care about.

In [89]:
combined_df.date = pd.to_datetime(combined_df.date, format='%d-%b-%Y')
combined_df.wdsp = pd.to_numeric(combined_df.wdsp, errors='coerce')
combined_wind_df = combined_df[['date', 'station name', 'wdsp', 'hm', 'ddhm', 'hg']]\
    .rename(columns={
        'wdsp': 'mean_wind_speed',
        'hm': 'highest_wind_speed',
        'ddhm': 'highest_wind_speed_direction',
        'hg': 'highest_gust'
    })
combined_wind_df.head()

,date,station name,mean_wind_speed,highest_wind_speed,highest_wind_speed_direction,highest_gust
0,2008-09-19,1775,0.1,0,10,0
1,2008-09-20,1775,0.1,0,10,0
2,2008-09-21,1775,0.1,0,0,0
3,2008-09-22,1775,0.1,0,0,0
4,2008-09-23,1775,0.1,0,10,0


Joining back to the station table gives us the locations and counties of each of the wind readings

In [95]:
cleaned_wind_df = combined_wind_df.merge(
    station_df[['station name', 'county', 'name', 'height(m)', 'latitude', 'longitude', 'easting', 'northing']], 
    on='station name', how='left')

cleaned_wind_df.to_csv('data/wind_data.csv', index=False)

In [96]:
cleaned_wind_df.head()

,date,station name,mean_wind_speed,highest_wind_speed,highest_wind_speed_direction,highest_gust,county,name,height(m),latitude,longitude,easting,northing
0,2008-09-19,1775,0.1,0,10,0,Wexford,JOHNSTOWN CASTLE 2,62,52.29778,-6.49667,302631,117200
1,2008-09-20,1775,0.1,0,10,0,Wexford,JOHNSTOWN CASTLE 2,62,52.29778,-6.49667,302631,117200
2,2008-09-21,1775,0.1,0,0,0,Wexford,JOHNSTOWN CASTLE 2,62,52.29778,-6.49667,302631,117200
3,2008-09-22,1775,0.1,0,0,0,Wexford,JOHNSTOWN CASTLE 2,62,52.29778,-6.49667,302631,117200
4,2008-09-23,1775,0.1,0,10,0,Wexford,JOHNSTOWN CASTLE 2,62,52.29778,-6.49667,302631,117200


We also want information on wind farms in Ireland. There is a dataset from 2022 with each of the wind farms in the 26 counties on data.gov.ie
https://data.gov.ie/dataset/wind-farms-in-ireland

Although this data is nearly 3 years out of date, this will give a good indication of the number and distribution of wind energy producers within the country. 

In [135]:
wind_farm_df = pd.read_csv('https://seaiopendata.blob.core.windows.net/wind/WindFarmsConnectedJune2022.csv')
wind_farm_df.to_csv("data/wind_farm_data.csv", index=False)

wind_farm_df.head()

,Windfarm_Name,DSO_TSO,Connection_Ref,County,Present_Status,Installed_Capacity__MW_,MEC__MW_,Gate,F110kV_Node_Name,Target_Connection,Date_of_Connection,Year_of_Connection,Nat_Grid_E__substation_,Nat_Grid_N__substation_,Type
0,Bellacorrick Wind Farm,DSO,DG955,Mayo,Connected,NaN,6.45,Pre-Gate,Bellacorick,01/10/1992,01/10/1992,1992,98670.0,321420.0,Wind
1,Corrie Mt.,DSO,DG961,Leitrim,Connected,NaN,4.80,Pre-Gate,Arigna,NaN,01/03/1997,1997,190043.0,320081.0,Wind
2,Mount Cronalaght,DSO,DG974,Donegal,Connected,NaN,4.98,Pre-Gate,Letterkenny,01/07/1997,01/07/1997,1997,186118.0,423981.0,Wind
3,Golagh (1),TSO,P25,Donegal,Connected,15.0,15.00,Pre-Gate,Golagh 110 kV,NaN,01/07/1997,1997,203554.0,382308.0,Wind
4,Drumlough Hill Wind Farm,DSO,DG920,Donegal,Connected,NaN,4.80,Pre-Gate,Trillick,01/08/1997,01/08/1997,1997,235797.0,439828.0,Wind


In [99]:
wind_farm_df[~pd.isna(wind_farm_df['Installed_Capacity__MW_'])]

,Windfarm_Name,DSO_TSO,Connection_Ref,County,Present_Status,Installed_Capacity__MW_,MEC__MW_,Gate,F110kV_Node_Name,Target_Connection,Date_of_Connection,Year_of_Connection,Nat_Grid_E__substation_,Nat_Grid_N__substation_,Type
3,Golagh (1),TSO,P25,Donegal,Connected,15.0,15.00,Pre-Gate,Golagh 110 kV,NaN,01/07/1997,1997,203554.0,382308.0,Wind
22,Kingsmountain (1),TSO,P38A,Sligo,Connected,25.0,23.75,Pre-Gate,Cunghill 110 kV,NaN,01/05/2003,2003,148479.0,328662.0,Wind
35,Meentycat (1),TSO,P38BDF,Donegal,Connected,72.4,70.96,Pre-Gate,Meentycat 110 kV,NaN,01/10/2004,2004,208891.0,402425.0,Wind
43,Ballywater (1),TSO,TG13,Wexford,Connected,31.5,31.50,Pre-Gate,Ballywater 110 kV,NaN,01/05/2005,2005,318720.0,144330.0,Wind
45,Booltiagh (1),TSO,TG07,Clare,Connected,19.5,19.45,Pre-Gate,Booltiagh 110 kV,NaN,01/08/2005,2005,115969.0,170581.0,Wind
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,Esk Wind Farm Phase 1,DSO,DG149a,Cork,Energised,0.0,5.40,Gate 3,BOGGERAGH,15/02/2020,16/10/2020,2020,145006.0,90090.0,Wind
309,Sorrell Island (Glenmore) WF Ext,DSO,DG1817,Clare,Energised,0.0,8.00,ECP-1,BOOLTIAGH,NaN,30/10/2020,2020,114719.0,169051.0,Wind
310,Carrigdangan (formerly Barnadivane),TSO,TG44,Cork,Connected,54.3,54.30,Gate 3,Dunmanway 110kV,NaN,01/01/2021,2021,134600.0,63300.0,Wind
311,Clogheravaddy Wind Farm (Phase 2),DSO,DG269B,Donegal,Connected,0.0,10.80,Gate 3,Binbane,NaN,01/01/2022,2022,182738.0,384378.0,Wind


This data gives us the county of each wind farm, the maximum export capacity in megawatts (MEC), the installed capacity (it's unclear what the difference between the two is, but it may be that MEC is what the station was contracted for and installed capacity is what was delivered, as some rows have higher MEC than installed, some have lower MEC than installed, and many are the same or have null Installed capacity)

We also have the eastings and northings of the farm's substation, which can be used as an approximation of the location of the farm itself. (it should be said that this is an approximation as the data provider notes that "a substation may or may not be located within the site of the wind farm it serves, it should be noted that the precise accuracy of this coordinate information cannot be assured").

In order to find the closest weather station to each substation we need to get the distances between each combination of them, then find the shortest distance.


In [128]:
weather_station_locations = cleaned_wind_df[['station name', 'easting', 'northing']].drop_duplicates()

substation_locations = wind_farm_df[['Nat_Grid_E__substation_', 'Nat_Grid_N__substation_']].rename(columns={
    'Nat_Grid_E__substation_': 'substation_easting',
    'Nat_Grid_N__substation_': 'substation_northing'}).drop_duplicates().reset_index(drop=True)\
.reset_index(names='substation_id')

In [129]:
substation_locations

,substation_id,substation_easting,substation_northing
0,0,98670.0,321420.0
1,1,190043.0,320081.0
2,2,186118.0,423981.0
3,3,203554.0,382308.0
4,4,235797.0,439828.0
...,...,...,...
275,275,128934.0,121526.0
276,276,144554.0,89720.0
277,277,145006.0,90090.0
278,278,134600.0,63300.0


In [133]:
import math
distance_df = weather_station_locations.merge(substation_locations, how='cross')
distance_df['distance'] = distance_df.apply(
    lambda row: math.sqrt((row['easting']-row['substation_easting'])**2 + 
                          (row['northing']-row['substation_northing'])**2), axis=1)

distance_df

,station name,easting,northing,substation_id,substation_easting,substation_northing,distance
0,1775,302631,117200,0,98670.0,321420.0,288627.611155
1,1775,302631,117200,1,190043.0,320081.0,232027.493856
2,1775,302631,117200,2,186118.0,423981.0,328161.333996
3,1775,302631,117200,3,203554.0,382308.0,283016.790302
4,1775,302631,117200,4,235797.0,439828.0,329477.783682
...,...,...,...,...,...,...,...
1955,375,273000,179500,275,128934.0,121526.0,155293.261386
1956,375,273000,179500,276,144554.0,89720.0,156712.549963
1957,375,273000,179500,277,145006.0,90090.0,156130.112842
1958,375,273000,179500,278,134600.0,63300.0,180712.478816


These distances are very high, it's possible that met eireann and the wind farm data providers are using different grid systems to encode locations eg. Irish national grid vs Irish Transverse Mercator, in which case we won't be able to resolve this without knowing the systems in use.

The wind farm data alludes to this - "Zipped collections of shapefiles are available in two spatial reference or coordinate systems:
1) Irish Transverse Mercator (ITM, EPSG:2157)
2) WGS 84 Web Mercator (EPSG:3857)", however this refers to the shapefiles but makes no mention of the csv.

It's also possible however, that these are the best estimates available to us, given that there are only 7 station datasets with the metrics we need it is likely that some of the wind farms will be quite far away from these given the sparse coverage. The two examples of our closest and farthest matches give two wind farms in Tipperary and Kerry both matching to the same weather station in Tipperary, which is not unimaginable as that is our only Munster-based dataset. This may be the more likely scenario, given that the shortest distances are around 2km and the largest are around 190km, which would be surprisingly coincidental if we were dealing with different grid systems - we would almost expect to see ludicrous distances if the systems were completely different.

In [140]:
distance_df.sort_values('distance').groupby('substation_id').first().sort_values('distance')

,station name,easting,northing,substation_easting,substation_northing,distance
substation_id,,,,,,
56,1475,199466,198376,201502.0,197587.0,2183.533146
57,1475,199466,198376,201175.0,196562.0,2492.243367
116,532,316900,243400,322271.0,241266.0,5779.411475
188,532,316900,243400,320128.0,250258.0,7579.719520
239,675,245200,311600,254409.0,307704.0,9999.224820
...,...,...,...,...,...,...
12,1475,199466,198376,116200.0,49200.0,170841.171068
106,1475,199466,198376,128280.0,41645.0,172139.632151
58,1475,199466,198376,115778.0,44892.0,174817.103282


In [143]:
cleaned_wind_df[cleaned_wind_df['station name'] == 1475].iloc[0]

date                            2008-02-06 00:00:00
station name                                   1475
mean_wind_speed                                11.1
highest_wind_speed                               24
highest_wind_speed_direction                    160
highest_gust                                     35
county                                    Tipperary
name                                        GURTEEN
height(m)                                        75
latitude                                     53.035
longitude                                  -8.00861
easting                                      199466
northing                                     198376
Name: 43074, dtype: object

In [145]:
wind_farm_df[(wind_farm_df['Nat_Grid_E__substation_'] == 201502) & (wind_farm_df['Nat_Grid_N__substation_'] == 197587)]


,Windfarm_Name,DSO_TSO,Connection_Ref,County,Present_Status,Installed_Capacity__MW_,MEC__MW_,Gate,F110kV_Node_Name,Target_Connection,Date_of_Connection,Year_of_Connection,Nat_Grid_E__substation_,Nat_Grid_N__substation_,Type
57,Carrig Wind Farm,DSO,DG12,Tipperary,Connected,NaN,2.55,PMOD,Dallow,NaN,29/05/2006,2006,201502.0,197587.0,Wind


In [147]:
cleaned_wind_df[cleaned_wind_df['station name'] == 1475].iloc[0]

date                            2008-02-06 00:00:00
station name                                   1475
mean_wind_speed                                11.1
highest_wind_speed                               24
highest_wind_speed_direction                    160
highest_gust                                     35
county                                    Tipperary
name                                        GURTEEN
height(m)                                        75
latitude                                     53.035
longitude                                  -8.00861
easting                                      199466
northing                                     198376
Name: 43074, dtype: object

In [146]:
wind_farm_df[(wind_farm_df['Nat_Grid_E__substation_'] == 55700) & (wind_farm_df['Nat_Grid_N__substation_'] == 78100)]


,Windfarm_Name,DSO_TSO,Connection_Ref,County,Present_Status,Installed_Capacity__MW_,MEC__MW_,Gate,F110kV_Node_Name,Target_Connection,Date_of_Connection,Year_of_Connection,Nat_Grid_E__substation_,Nat_Grid_N__substation_,Type
154,Knockaneden Wind Farm,DSO,DG118,Kerry,Connected,NaN,9.0,Gate 2,Oughtragh,01/09/2012,14/09/2012,2012,55700.0,78100.0,Wind


In [148]:
cleaned_wind_df.county.drop_duplicates()

0          Wexford
5947         Meath
12758       Dublin
43074    Tipperary
49248    Westmeath
67930        Cavan
75666       Carlow
Name: county, dtype: object